In [ ]:
version = "7.10.8"

In [ ]:
! wget -q https://github.com/PelicanPlatform/pelican/releases/download/v{version}/pelican_Linux_x86_64.tar.gz
! tar -zxvf pelican_Linux_x86_64.tar.gz
! rm pelican_Linux_x86_64.tar.gz*

In [ ]:
import os
os.environ["PATH"] += f":{os.getcwd()}/pelican-{version}"

In [ ]:
from pelicanfs.core import PelicanFileSystem, PelicanMap
import fsspec
from pathlib import Path

In [ ]:
federation = "osg-htc.org"
namespace = "/ndp/burnpro3d"
pelfs = PelicanFileSystem(f"pelican://{federation}")

In [ ]:
for obj in pelfs.ls(namespace):
    print(obj)

In [ ]:
fpath = "osdf:///" + "/ndp/burnpro3d/birch.png"

from PIL import Image
from IPython.display import display

with fsspec.open(str(fpath), "rb") as fi:
    print(fi)
    im = Image.open(fi)
    im.load()

print(im.format, im.size, im.mode)

if max(im.size) > 1000:
    print("Resizing")
    size_x, size_y = im.size
    im = im.resize((size_x // 8, size_y // 8))

display(im)

In [ ]:
fpath = Path("osdf:///" + "/ndp/burnpro3d/TLS_scans.zip")

! pelican object get -f {federation} {fpath} {fpath.name}
! python -m zipfile -e {fpath.name} .
! rm {fpath.name}
! ls -ltrh

In [ ]:
import trimesh
mesh = trimesh.load("TLS_scans/original_ply/Area_5_05_20_1756.ply")
print(mesh)

In [ ]:
import plotly.graph_objects as go
import numpy as np
%matplotlib widget

# Grab a random subset of points
everything = len(mesh.vertices)
max_points = min(50_000, everything)
print(f"Max points: {max_points}")
subset = np.random.randint(low=0, high=everything, size=max_points)
points = mesh.vertices[subset]

# Create a scatter plot for the points using Plotly
scatter = go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(size=2, color='blue')
)

# Set up the layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    ),
    width=800,
    height=800,
)

# Create the figure and display it
fig = go.Figure(data=[scatter], layout=layout)
fig.show()